In [2]:
# Matthew Berning
# import dependancies
import pandas as pd
from bs4 import BeautifulSoup # as bs
import requests
from requests import get
from splinter import Browser
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import os
import re
import string
import urllib

In [2]:
# URL of page to be scraped
mars_news_url = 'https://mars.nasa.gov/news/'
#  Retrieve page with the requests module
response = requests.get(mars_news_url)
# Create BeautifulSoup object; parse with 'lxml'
soup_level0 = BeautifulSoup(response.text, 'lxml')
#test
# print(soup) 

In [3]:
#get all article titles (that are able to be scraped)
article_titles = soup_level0.find_all('div', class_='content_title')
# print(article_titles)

#retrieve just the first article's title 
#(there is a problem with the BS codebase, it is somehow only scraping the infromation from the scrolling carosel on the very bottom of the page)

first_article = soup_level0.find('div', class_='content_title').get_text()
print(first_article)



NASA's InSight Places First Instrument on Mars




In [4]:
#get article description
first_article_description = soup_level0.find('div', class_='rollover_description_inner').get_text()
print(first_article_description)


In deploying its first instrument onto the surface of Mars, the lander completes a major mission milestone.



In [5]:
#url for featured image to be scraped
space_images_url = 'https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars'

# create a new chrome session
driver = webdriver.Chrome()
driver.implicitly_wait(30)
driver.get(space_images_url)

#use selenium to click button 'full image'
python_button = driver.find_element_by_id('full_image') 
python_button.click() #click  link

#once 'full image' is opened click 'more info' button to get to ACTUAL page with the full res image
python_button = driver.find_element_by_partial_link_text('more info')
python_button.click() #click link

#on the new page use beautifulsoup to get page source
soup_level1=BeautifulSoup(driver.page_source, 'lxml')

#test the soup to find full resloution image
# soup_level1.body.find_all('img', class_ ='main_image')

#pull out image 'src' by class and 'img' tag
img_find_src = [i['src'] for i in soup_level1.findAll('img', {'class': 'main_image'})]
# from STACKOVERFLOW: https://stackoverflow.com/questions/8289957/python-2-7-beautiful-soup-img-src-extract 

#cast soup.find to string
img_str = str(img_find_src)
#need to remove brackets and appostrophies from string generated by above
whitelist = string.ascii_letters + string.digits + '/' + '_' + '.'
img_url_end = ''
for char in img_str:
    if char in whitelist:
        img_url_end += char
    else:
        img_url_end += ''
#from STACKOVERFLOW: https://stackoverflow.com/questions/875968/how-to-remove-symbols-from-a-string-with-python        
       
img_url_start = 'https://www.jpl.nasa.gov' #begining of url
featured_img_url = (img_url_start + img_url_end) #FEATURED IMAGE FULL RESOLOUTION URL
print(featured_img_url)

https://www.jpl.nasa.gov/spaceimages/images/largesize/PIA14400_hires.jpg


In [6]:
#the 'featured image' from the home work link is not always mars, even though it should be, 
#so it will just be a featured space image that will be put at the bottom of the finished webpage along with the other mars stuff

#capture image title
featured_image_title = soup_level1.find('h1', class_="article_title").text
#test
# print(featured_image_title)

#capture using beautiful soup all paragraph elements 
paragraphs = soup_level1.find_all('p')

#create empty list to contain p elements
paragraph_list = []

#iterate through retrieved paragraphs and conditinally append to paragraph list
for paragraph in paragraphs:
    p = [] #make empty list to contain text from each paragraph element individually
    p.append(paragraph.text)
    if any("Mission:" in s for s in p): #from STACKOVERFLOW: https://stackoverflow.com/questions/4843158/check-if-a-python-list-item-contains-a-string-inside-another-string
        break #end the loop when infromation from the 'image details' section of p elements is entered
    else:
        paragraph_list.append(paragraph) #append each paragraph to the paragraph list
#test
print(featured_image_title)

j = 1

for i in range(len(paragraph_list)):
    print("------------")
    print("paragraph No." + str(j)+":")
    print("------------")
    print(paragraph_list[i].text)
    j+=1
    


				The Case of the Warped Galactic Ring			  
------------
paragraph No.1:
------------

------------
paragraph No.2:
------------

------------
paragraph No.3:
------------
In a strange twist of science, astronomers using the Herschel Space Observatory have discovered that a suspected ring at the center of our galaxy is warped for reasons they cannot explain. This image from Herschel, an infrared European Space Agency-led mission with important NASA contributions, reveals the ring with greater clarity than ever before. It can be seen as the yellow loop that appears to have two lobes (Figure 1). In fact, the ring, which is a collection of very dense and cold gas and dust, is twisted so that part of it rises above and below the plane of our Milky Way galaxy. 
------------
paragraph No.4:
------------
Previous observations had revealed portions of the ring. Herschel sees long-wavelength infrared light, which can penetrate through the murky region at the center of our galaxy, allowing He

In [7]:
# URL to get mars weather from twitter
mars_weather_url = 'https://twitter.com/marswxreport?lang=en'
#  Retrieve page with the requests module
response = requests.get(mars_weather_url)
# Create BeautifulSoup object; parse with 'lxml'
soup_level2 = BeautifulSoup(response.text, 'lxml')
 
#make empty list for weather tweets
weather_tweets = []

#iterate through soup and get tweet text
for li in soup_level2.find_all("li", {"data-item-type": "tweet"}):
    #make empty lists for all available tweets on page
    all_tweets = []
    text_p = li.find("p", class_="tweet-text")
    if text_p is not None:
        all_tweets.append(text_p.get_text()) #from STACKOVERFLOW: https://stackoverflow.com/questions/51653786/how-to-get-all-tweets-from-a-public-account-by-beautifulsoup
        #iterate through list of tweets and keep only ones that start with 'Sol'
        for i in all_tweets: 
            t = []
            t.append(i)
            if any("Sol" in s for s in t):
                weather_tweets.append(i)

#assign just the first *most current* one to variable 
current_mars_weather = str(weather_tweets[0])

print(current_mars_weather)

Sol 2298 (2019-01-23), high -9C/15F, low -71C/-95F, pressure at 8.17 hPa, daylight 06:46-18:55pic.twitter.com/HNaq1Rjsoq


In [22]:
# URL for mars geophysical facts
url = 'https://space-facts.com/mars/'
#read html with pandas and push into list
get_table = pd.read_html(url)
#test
# get_table
#build dataframe from table using index, because apparently you have to
df = get_table[0]
#rename columns 
df.columns = ['Info Type:', 'Data:']
#test
# df
#use pandas to turn table into an html string
mars_facts_html_table = df.to_html()
#test
mars_facts_html_table

'<table border="1" class="dataframe">\n  <thead>\n    <tr style="text-align: right;">\n      <th></th>\n      <th>Info Type:</th>\n      <th>Data:</th>\n    </tr>\n  </thead>\n  <tbody>\n    <tr>\n      <th>0</th>\n      <td>Equatorial Diameter:</td>\n      <td>6,792 km</td>\n    </tr>\n    <tr>\n      <th>1</th>\n      <td>Polar Diameter:</td>\n      <td>6,752 km</td>\n    </tr>\n    <tr>\n      <th>2</th>\n      <td>Mass:</td>\n      <td>6.42 x 10^23 kg (10.7% Earth)</td>\n    </tr>\n    <tr>\n      <th>3</th>\n      <td>Moons:</td>\n      <td>2 (Phobos &amp; Deimos)</td>\n    </tr>\n    <tr>\n      <th>4</th>\n      <td>Orbit Distance:</td>\n      <td>227,943,824 km (1.52 AU)</td>\n    </tr>\n    <tr>\n      <th>5</th>\n      <td>Orbit Period:</td>\n      <td>687 days (1.9 years)</td>\n    </tr>\n    <tr>\n      <th>6</th>\n      <td>Surface Temperature:</td>\n      <td>-153 to 20 °C</td>\n    </tr>\n    <tr>\n      <th>7</th>\n      <td>First Record:</td>\n      <td>2nd millennium 

In [ ]:
#MARS HEMISPHERES PLACE HOLDER
#UNABLE TO SCRAPE BECAUSE THE GOVERNMENT IS SHUT
#
#
#
#
#
#
#
#
#
#
#END PLACE HOLDER
